# PTT 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 印出最新文章的「作者」「標題」「時間」
* ② 印出第一頁所有文章的「作者」「標題」「時間」


### ① 印出最新文章的「作者」「標題」「時間」

In [17]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

url = 'https://www.ptt.cc/bbs/NBA/index.html'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html5lib')

newtag=soup.find('a',string='最新') #根據string找到標籤
newlink='https://www.ptt.cc/'+newtag['href'] #獲得最新文章頁面的超連結後，記得組成完整網址

#獲取最新文章所在頁面的原始碼 
res2=BeautifulSoup(requests.get(newlink).text)

#將時間整數轉成日期
def TransTime(n):
        timeint=int(n) #字串轉成整數資料，localtime只接受整數
        times=time.localtime(timeint) #轉換日期
        timeresult=time.strftime('%Y-%m-%d %H:%M:%S',times) #格式化
        return timeresult
    
#找出最新文章
tags=res2.find_all('div',class_="r-ent")

titlelist=[]
authorlist=[]
timelist=[]
for tag in tags:
    try:
        timedata=tag.a['href'].split('.')[1] #從標籤 a的屬性 href取出時間整數值(資料型態:字串)
        returntime=TransTime(timedata) #從函式回傳一個轉換後的時間值
        timelist.append(returntime) #將此回傳值加到時間列表
        authortag=tag.find('div',class_="author") #找作者標籤
        authorlist.append(authortag.text) #建立作者list
        titlelist.append(tag.a.string) #建立文章標題list
        
    except:
        continue
        
#建立dataframe
df=pd.DataFrame({'時間':timelist,'作者':authorlist,'標題':titlelist})
print('最新文章時間',df['時間'].max()) #找出最新時間
n=0
for i in df.iloc[:,0]: #對時間欄內所有時間比對
    n+=1
    if i ==df['時間'].max(): #若等於最新時間，輸出那一行(時間 &作者 &標題)
        print(df.iloc[n-1,:])

最新文章時間 2020-04-13 23:59:40
時間          2020-04-13 23:59:40
作者                       Gotham
標題    [新聞] 保羅籲球員藉機學理財 專家稱鄧肯與賈奈特
Name: 8, dtype: object


### ② 印出第一頁所有文章的「作者」「標題」「時間」

In [19]:
#由看板任一頁url獲得最新一頁url
def newestPage(url):
    r=requests.get(url)
    res=BeautifulSoup(r.text)
    newtag=res.find('a',string='最新') #根據string找到標籤
    newlink='https://www.ptt.cc/'+newtag['href'] #獲得最新文章頁面的超連結後，記得組成完整網址
    return newlink
    
# newlink=newestPage('https://www.ptt.cc/bbs/NBA/index6501.html') #任一頁url丟入函式，獲得最新頁url

In [33]:
#建立函式:從最新一頁逐漸往上一頁搜尋文章
def PageData(link):
    r=BeautifulSoup(requests.get(link).text) #解析最新頁htnl網頁碼
    #搜尋標籤   
    rtags=r.find_all('div',class_="r-ent")
    titlelist=[]
    authorlist=[]
    timelist=[]

    for rtag in rtags:
        try:
            titlelist.append(rtag.a.text) #建立文章標題list
            authortag=rtag.find('div',class_="author") #找作者標籤
            authorlist.append(authortag.text) #建立作者list
            timedata=rtag.a['href'].split('.')[1] #從標籤 a的屬性 href取出時間整數值(資料型態:字串)
            returntime=TransTime(timedata) #從函式回傳一個轉換後的時間值
            timelist.append(returntime) #將此回傳值加到時間列表

        except:
            continue

    #建立dataframe印出文章資料
    print(pd.DataFrame({'作者':authorlist,'時間':timelist,'標題':titlelist}))
    
    #回傳上一頁url
    newtag=r.find('a',string='‹ 上頁')
    lastpagelink='https://www.ptt.cc/'+newtag['href'] #獲得上一頁頁面的超連結後，記得組成完整網址
    return lastpagelink

#呼叫函式印出最新一頁內容
PageData(newlink)

              作者                   時間                                    標題
0      fukawa947  2020-04-13 17:10:28               [新聞] 為何交易掉羅素？勇士老闆：因為威金斯
1          doepe  2020-04-13 18:00:49                [新聞] 柯瑞超狂射程　出手總長超兩座聖母峰
2           Yui5  2020-04-13 18:15:58               [花邊] Cuban：疫情讓我想要去競選總統了
3       asdf1256  2020-04-13 19:55:49              [新聞] 確診後收死亡威脅　戈貝爾懺悔：亂摸麥克
4    love1500274  2020-04-13 21:26:06  [專欄] The Players' Tribune - 衝破黑暗-林書豪
5       Intelnet  2020-04-13 21:41:50              [花邊] 皮朋喊是名人堂遺珠 庫寇奇：隨喬丹奪冠
6     thnlkj0665  2020-04-13 22:28:41          [花邊] 護士防護服下穿自己球衣 柯瑞視訊來電親自給鼓勵
7       fatotaku  2020-04-13 22:49:40    Re: [花邊] 陣容對比圖：最近的5位MVP VS 最近的FMVP
8         Gotham  2020-04-13 23:59:40             [新聞] 保羅籲球員藉機學理財 專家稱鄧肯與賈奈特
9       ghost069  2020-04-14 00:04:35                      [情報] 拓荒者錄影助理意外去世
10       Bainite  2020-04-14 00:06:52                         [情報] 公牛GM被開除了
11         Vedan  2019-05-24 19:43:14                           [公告] 板規v6.8
12  qazwsx87

'https://www.ptt.cc//bbs/NBA/index6501.html'

In [34]:
#使用迴圈可抓取多頁
count=0
link=newlink
while count<2: #想印出幾頁就跑幾次迴圈
    link=PageData(link) #將回傳的上一頁url做為新的url丟進函式獲得另一頁的資料
    count+=1

              作者                   時間                                    標題
0      fukawa947  2020-04-13 17:10:28               [新聞] 為何交易掉羅素？勇士老闆：因為威金斯
1          doepe  2020-04-13 18:00:49                [新聞] 柯瑞超狂射程　出手總長超兩座聖母峰
2           Yui5  2020-04-13 18:15:58               [花邊] Cuban：疫情讓我想要去競選總統了
3       asdf1256  2020-04-13 19:55:49              [新聞] 確診後收死亡威脅　戈貝爾懺悔：亂摸麥克
4    love1500274  2020-04-13 21:26:06  [專欄] The Players' Tribune - 衝破黑暗-林書豪
5       Intelnet  2020-04-13 21:41:50              [花邊] 皮朋喊是名人堂遺珠 庫寇奇：隨喬丹奪冠
6     thnlkj0665  2020-04-13 22:28:41          [花邊] 護士防護服下穿自己球衣 柯瑞視訊來電親自給鼓勵
7       fatotaku  2020-04-13 22:49:40    Re: [花邊] 陣容對比圖：最近的5位MVP VS 最近的FMVP
8         Gotham  2020-04-13 23:59:40             [新聞] 保羅籲球員藉機學理財 專家稱鄧肯與賈奈特
9       ghost069  2020-04-14 00:04:35                      [情報] 拓荒者錄影助理意外去世
10       Bainite  2020-04-14 00:06:52                         [情報] 公牛GM被開除了
11         Vedan  2019-05-24 19:43:14                           [公告] 板規v6.8
12  qazwsx87

### ③ 試著爬爬看其他版的文章

In [36]:
#ptt Stock
count=0
link='https://www.ptt.cc/bbs/Stock/index.html'
while count<2:
    link=PageData(link) 
    count+=1

           作者                   時間                                          標題
0        j547  2020-04-13 23:25:09                            Re: [標的] 道瓊 伊歐玟空
1        Su22  2020-04-13 23:53:23                    [新聞] 陸企裁員潮 海信集團驚傳裁員萬人坦承形
2     dust777  2020-04-14 00:06:09                  [標的] 9910 豐泰 多 / 5871 中租 多
3       IanLi  2015-01-25 23:18:25                               [公告] 精華區導覽Q&A
4     eyespot  2018-11-03 21:36:51  [公告] Stock 板規V2.6             (2019/06/20)
5   noldorelf  2020-03-28 22:48:52                                   [公告] 版規修正
6     eyespot  2020-04-04 23:45:37                              [公告] 股票板板主徵選公告
7  vendan5566  2020-04-13 14:00:02                        [閒聊] 2020/04/13 盤後閒聊
              作者                   時間                                  標題
0     steward135  2020-04-13 20:40:35               [其他] 109年04月13日信用交易統計
1   fish19831012  2020-04-13 20:45:04        [新聞]國安基金靠10億元殺進殺出股市？蘇建榮不只這規模
2          werqq  2020-04-13 20:47:01           Re: [請益] 如果爆發社區感染是否